In [71]:
from pyspark.sql import SparkSession
import numpy as np
from pyspark import SparkContext
from pyspark.sql import HiveContext
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import *
import datetime

In [72]:
start_time = datetime.datetime.now()

In [73]:
spark = SparkSession \
    .builder \
    .appName("Assignment2 - COMP5349 - Stage 2") \
    .getOrCreate()
sqlContext = HiveContext(sc)

In [74]:
awsData = "s3://amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"

In [75]:
#Read music data from csv file stored in AWS
musicData = spark.read.csv(awsData,header=True,sep='\t')
musicData.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)

In [76]:
#Select only the required fields
requiredData = musicData.select('customer_id','product_id','star_rating','review_id','review_body')
requiredData = requiredData.repartition(100)
#requiredData.show(5)

In [77]:
#Split the review body based on '.' or '?'
reviewBody=requiredData.withColumn('Number_Of_Sentences', size(split(col("review_body"), r"\.|\?")) - 1).cache()
reviewBody.show(5)

+-----------+----------+-----------+--------------+--------------------+-------------------+
|customer_id|product_id|star_rating|     review_id|         review_body|Number_Of_Sentences|
+-----------+----------+-----------+--------------+--------------------+-------------------+
|   15245952|B0012GMUY0|          5|R3DD1TI7T7AZU5|Allman,Trucks and...|                  2|
|     372935|B00030B9FW|          5| RF7XIGVVGFXEV|Wonderful tv jazz...|                  3|
|   50736950|B00000IWVW|          3|R1329J6F9LDZ3U|this set isn't ba...|                  5|
|   49837680|B00001X590|          5|R142PTIHMM25N9|I have the old on...|                  8|
|   39168843|B00005CEP5|          5|R2HFD37NLHB9P9|The songs are sun...|                  8|
+-----------+----------+-----------+--------------+--------------------+-------------------+
only showing top 5 rows

In [78]:
#Remove fields with less than 2 number of sentences
reviewWithMoreThanTwoSentence = reviewBody.filter(col("Number_Of_Sentences") >= 2)
reviewWithMoreThanTwoSentence.cache()
reviewWithMoreThanTwoSentence.count()

3875475

In [79]:
reviewWithMoreThanTwoSentence.show(5)

+-----------+----------+-----------+--------------+--------------------+-------------------+
|customer_id|product_id|star_rating|     review_id|         review_body|Number_Of_Sentences|
+-----------+----------+-----------+--------------+--------------------+-------------------+
|   15245952|B0012GMUY0|          5|R3DD1TI7T7AZU5|Allman,Trucks and...|                  2|
|     372935|B00030B9FW|          5| RF7XIGVVGFXEV|Wonderful tv jazz...|                  3|
|   50736950|B00000IWVW|          3|R1329J6F9LDZ3U|this set isn't ba...|                  5|
|   49837680|B00001X590|          5|R142PTIHMM25N9|I have the old on...|                  8|
|   39168843|B00005CEP5|          5|R2HFD37NLHB9P9|The songs are sun...|                  8|
+-----------+----------+-----------+--------------+--------------------+-------------------+
only showing top 5 rows

In [80]:
#Group by customer_id to get total number of reviews published
users = reviewWithMoreThanTwoSentence.groupBy("customer_id").count().sort(col("count"))
users.show(5)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   32370558|    1|
|   35294520|    1|
|   50991029|    1|
|    6209942|    1|
|   51033615|    1|
+-----------+-----+
only showing top 5 rows

In [81]:
#caching to reduce processing time
users.cache()
users.count()

1598204

In [82]:
#Calculate median
userMedian = users.approxQuantile("count", [0.5], 0)
print("User median: ", userMedian)

('User median: ', [1.0])

In [83]:
#Remove users with less than median number of reviews published
usersWithMoreThanMedianReviews = users.where(col("count") > userMedian[0])
#usersWithMoreThanMedianReviews.show(5)

In [84]:
usersWithMoreThanMedianReviews = usersWithMoreThanMedianReviews.select(col("customer_id").alias("user_customer_id"), col("count"))

In [85]:
#Join filtered data with reviewWithMoreThanTwoSentence 
innerJoin = reviewWithMoreThanTwoSentence.join(usersWithMoreThanMedianReviews, reviewWithMoreThanTwoSentence.customer_id == usersWithMoreThanMedianReviews.user_customer_id, "inner")
innerJoin.cache()
innerJoin.count()
#innerJoin.show(5)

2747750

In [86]:
innerJoin.rdd.getNumPartitions()

200

In [87]:
#Group by producta_id to get total number of reviews received
product = innerJoin.groupBy("product_id").count().sort(col("count"))
product.cache()
product.count()
#product.show(5)

593788

In [88]:
#Calucluate median
prodMedian = product.approxQuantile("count", [0.5], 0)
print("Product median", prodMedian)

('Product median', [1.0])

In [89]:
product = product.select(col("product_id").alias("prod_id"), col("count").alias("product_count"))
#product.show(5)

In [90]:
#Remove products with less than median number of reviews received
productsWithMoreThanMedianReviews = product.where(col("product_count") > prodMedian[0])
#productsWithMoreThanMedianReviews.show(5)

In [91]:
productInnerJoin = innerJoin.join(productsWithMoreThanMedianReviews, innerJoin.product_id == productsWithMoreThanMedianReviews.prod_id, "inner")
#productInnerJoin.show(5)

In [92]:
#Calculate users number of sentences in the reviews they have published
topTenUsers = productInnerJoin.groupBy("customer_id").agg(F.collect_set("review_id"), F.collect_list("Number_Of_Sentences"))
topTenUsers.cache()
topTenUsers.count()
#topTenUsers.show(5)

463099

In [93]:
def media_calc(list_sent_count):
    median_val=np.median(list_sent_count)
    return int(median_val)

In [94]:
#Remove users with less than median number of sentences they have published
median_calc_udf=udf(media_calc, IntegerType())
topUsers_withMedian=topTenUsers.withColumn("median_val", median_calc_udf(col("collect_list(Number_Of_Sentences)")))
topUsers_withMedian.show(5)

+-----------+----------------------+---------------------------------+----------+
|customer_id|collect_set(review_id)|collect_list(Number_Of_Sentences)|median_val|
+-----------+----------------------+---------------------------------+----------+
|   10010810|  [R53XG9B98548F, R...|                          [7, 10]|         8|
|   10021653|      [R2VQWQVRKY22C1]|                              [7]|         7|
|   10071733|  [R3V8SWVNXRMQE6, ...|             [5, 23, 36, 27, 9...|        16|
|   10077989|  [R35R5UJZD2S3KL, ...|                          [10, 3]|         6|
|   10081213|  [R1KXZ2BUVLGDJ3, ...|                    [7, 21, 9, 7]|         8|
+-----------+----------------------+---------------------------------+----------+
only showing top 5 rows

In [95]:
#Limits it to top 10 users
topTenUsers = topUsers_withMedian.select(col("customer_id"), col("median_val")).orderBy("median_val", ascending=False)
topTenUsers = topTenUsers.limit(10)
topTenUsers.show()

+-----------+----------+
|customer_id|median_val|
+-----------+----------+
|   40611822|       810|
|   50611589|       708|
|   15741015|       648|
|   31354040|       427|
|   50595705|       317|
|   25628286|       298|
|   28331791|       257|
|   32626976|       255|
|   51865782|       252|
|   35846990|       239|
+-----------+----------+

In [96]:
#Calculate products number of sentences in the reviews they have received
topTenProducts = productInnerJoin.groupBy("product_id").agg(F.collect_set("review_id"), F.collect_list("Number_Of_Sentences"))
topTenProducts.cache()
topTenProducts.count()
#topTenProducts.show(10)

293133

In [97]:
#Remove products with less than median number of sentences they have received
median_calc_udf=udf(media_calc, IntegerType())
topTenProducts_withMedian=topTenProducts.withColumn("median_val", median_calc_udf(col("collect_list(Number_Of_Sentences)")))
#topTenProducts_withMedian.show(5)

In [98]:
#Limit it to top 10 products
topTenProducts = topTenProducts_withMedian.select(col("product_id"), col("median_val")).orderBy("median_val", ascending=False)
topTenProducts = topTenProducts.limit(10)
topTenProducts.show()

+----------+----------+
|product_id|median_val|
+----------+----------+
|B000ICM710|       411|
|B00T7TYTCK|       400|
|B000GUK040|       297|
|B000BFNZ6U|       272|
|B00PYI2VY6|       250|
|B0002IJNGC|       249|
|B0002MQ7CQ|       246|
|B00HSP0P0U|       242|
|B000RY431G|       241|
|B000LGN00K|       239|
+----------+----------+

In [99]:
end_time = datetime.datetime.now()

In [100]:
print("Execution time: ", str(end_time-start_time))

('Execution time: ', '0:01:02.512454')